In [1]:
import cv2 as cv
import numpy as np
from tqdm import tqdm_notebook
from glob import glob
from os.path import join

In [2]:
def bb_intersection_over_union(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
 
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 
    boxAArea = (boxA[2] + 1) * (boxA[3] + 1)
    boxBArea = (boxB[2] + 1) * (boxB[3] + 1)
 
    iou = interArea / float(boxAArea + boxBArea - interArea)
    
    return iou

In [3]:
Images = np.load('Images.npy')
Coords = np.load('Coords.npy')
Labels = np.load('Labels.npy')

In [4]:
original_coords = {}
for i in range(len(Images)):
    if Images[i] in original_coords:
        original_coords[Images[i]].append(Coords[i])
    else:
        original_coords[Images[i]] = [Coords[i]]

In [5]:
BASEROOT = 'data'
FROMNSET, TONSET = (1, 2)

CANDIDATES = ['set{0:02d}'.format(nset) 
              for nset in range(FROMNSET, TONSET)]

all_images = []

for candidate in tqdm_notebook(CANDIDATES):
    setsimages = glob(join(BASEROOT, candidate, '*'))
    for fimage in setsimages:
        all_images = all_images + glob(join(fimage, '*'))

In [6]:
face_cascade = cv.CascadeClassifier('haarcascade_fullbody.xml')
calculated = {}
for picture in tqdm_notebook(all_images):
    grayscale_image = cv.cvtColor(cv.imread(picture), cv.COLOR_BGR2GRAY)
    detected_faces = face_cascade.detectMultiScale(grayscale_image)
    if detected_faces == ():
        calculated[picture] = []
    else:
        calculated[picture] = detected_faces
    if picture not in original_coords:
        original_coords[picture] = []

C:\Users\David\.conda\envs\dango\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


In [7]:
'''
print(calculated['data\\set01\\pictures001\\I01441.jpg'])
print(original_coords['data\\set01\\pictures001\\I01441.jpg'])
'''

"\nprint(calculated['data\\set01\\pictures001\\I01441.jpg'])\nprint(original_coords['data\\set01\\pictures001\\I01441.jpg'])\n"

In [8]:
'''
test_picture = 'data\\set01\\pictures001\\I01441.jpg'
original_image = cv.imread(test_picture)
grayscale_image = cv.cvtColor(original_image, cv.COLOR_BGR2GRAY)
for (column, row, width, height) in calculated[test_picture]:
    cv.rectangle(
        original_image,
        (column, row),
        (column + width, row + height),
        (0, 255, 0),
        2
    )
for (column, row, width, height) in original_coords[test_picture]:
    cv.rectangle(
        original_image,
        (column, row),
        (column + width, row + height),
        (0, 0, 255),
        2
    )
cv.imshow('Image', original_image)
cv.waitKey(0)
cv.destroyAllWindows()
'''

"\ntest_picture = 'data\\set01\\pictures001\\I01441.jpg'\noriginal_image = cv.imread(test_picture)\ngrayscale_image = cv.cvtColor(original_image, cv.COLOR_BGR2GRAY)\nfor (column, row, width, height) in calculated[test_picture]:\n    cv.rectangle(\n        original_image,\n        (column, row),\n        (column + width, row + height),\n        (0, 255, 0),\n        2\n    )\nfor (column, row, width, height) in original_coords[test_picture]:\n    cv.rectangle(\n        original_image,\n        (column, row),\n        (column + width, row + height),\n        (0, 0, 255),\n        2\n    )\ncv.imshow('Image', original_image)\ncv.waitKey(0)\ncv.destroyAllWindows()\n"

In [9]:
score = 0
for picture in tqdm_notebook(all_images):
    if len(calculated[picture]) > 0:
        for item in calculated[picture]:
            if len(original_coords[picture]) == 0:
                break
            for original_item in original_coords[picture]:
                score = score + bb_intersection_over_union(item, original_item)
score = score/len(all_images)
print(score)


0.047956296687984006
